In [ ]:
# 日本百名湯のうち、wikipediaに記事のある温泉のリスト
# -----------------------------------------------------------------------------
# 目的:
#   ・後続の NLP / 検索（例: TF-IDF, Elasticsearch, Discovery）に用いる学習/評価コーパスとして、
#     Wikipedia（日本語）から「日本百名湯」に関する本文テキストを収集する。
# 背景・理論的補足:
#   ・Wikipedia は汎用百科事典ゆえ、用語説明や歴史・地理・泉質など多様な語彙が混在する。
#     これはベクトル化（BoW/TF-IDF/word2vec/BERT 等）や同義語展開の効果検証に向く。
#   ・語彙分布は Zipf 則に従いやすく、上位語の出現頻度が高くなるため、ベクトル化時は
#     stopword/品詞フィルタや max_df/min_df 等の抑制が理にかなう。
#   ・Wikipedia 由来のデータは CC BY-SA / GFDL のライセンスに従う必要がある。派生物利用時は注意。
# 運用上の注意:
#   ・`wikipedia` パッケージは内部で MediaWiki API を叩く。回数/速度が多い場合はレート制御を検討。
#   ・auto_suggest=False により題名の曖昧補正を無効化しているため、存在しない/曖昧な項目では
#     例外（DisambiguationError/PageError）が発生する。実運用では try/except で握るのが望ましい。
#   ・本文は wiki マークアップを含むプレーンテキスト（セクション見出し "== ==" など）で返る。
#     後段の形態素解析では不要記号の除去等の前処理が有効（例: 正規化・記号除去）。
title_list = [
    "菅野温泉",
    "養老牛温泉",
    "定山渓温泉",
    "登別温泉",
    "洞爺湖温泉",
    "ニセコ温泉郷",
    "朝日温泉 (北海道)",
    "酸ヶ湯温泉",
    "蔦温泉",
    "花巻南温泉峡",
    "夏油温泉",
    "須川高原温泉",
    "鳴子温泉郷",
    "遠刈田温泉",
    "峩々温泉",
    "乳頭温泉郷",
    "後生掛温泉",
    "玉川温泉 (秋田県)",
    "秋ノ宮温泉郷",
    "銀山温泉",
    "瀬見温泉",
    "赤倉温泉 (山形県)",
    "東山温泉",
    "飯坂温泉",
    "二岐温泉",
    "那須温泉郷",
    "塩原温泉郷",
    "鬼怒川温泉",
    "奥鬼怒温泉郷",
    "草津温泉",
    "伊香保温泉",
    "四万温泉",
    "法師温泉",
    "箱根温泉",
    "湯河原温泉",
    "越後湯沢温泉",
    "松之山温泉",
    "大牧温泉",
    "山中温泉",
    "山代温泉",
    "粟津温泉",
    "奈良田温泉",
    "西山温泉 (山梨県)",
    "野沢温泉",
    "湯田中温泉",
    "別所温泉",
    "中房温泉",
    "白骨温泉",
    "小谷温泉",
    "下呂温泉",
    "福地温泉",
    "熱海温泉",
    "伊東温泉",
    "修善寺温泉",
    "湯谷温泉 (愛知県)",
    "榊原温泉",
    "木津温泉",
    "有馬温泉",
    "城崎温泉",
    "湯村温泉 (兵庫県)",
    "十津川温泉",
    "南紀白浜温泉",
    "南紀勝浦温泉",
    "湯の峰温泉",
    "龍神温泉",
    "奥津温泉",
    "湯原温泉",
    "三朝温泉",
    "岩井温泉",
    "関金温泉",
    "玉造温泉",
    "有福温泉",
    "温泉津温泉",
    "湯田温泉",
    "長門湯本温泉",
    "祖谷温泉",
    "道後温泉",
    "二日市温泉 (筑紫野市)",
    "嬉野温泉",
    "武雄温泉",
    "雲仙温泉",
    "小浜温泉",
    "黒川温泉",
    "地獄温泉",
    "垂玉温泉",
    "杖立温泉",
    "日奈久温泉",
    "鉄輪温泉",
    "明礬温泉",
    "由布院温泉",
    "川底温泉",
    "長湯温泉",
    "京町温泉",
    "指宿温泉",
    "霧島温泉郷",
    "新川渓谷温泉郷",
    "栗野岳温泉",
]

# wikipediaの記事の読み取り
# -----------------------------------------------------------------------------
# 役割:
#   ・`wikipedia` ライブラリで各タイトルの記事本文を取得し、後続処理で扱いやすい dict に整形。
# 実装ポイント:
#   ・`set_lang("ja")` により日本語版 API を利用。ローカライズされた語彙・表記を得られる。
#   ・進捗可視化のために (index+1, title) を逐次 print。
#   ・`app_id` は 1 始まりの連番。外部システム（Elasticsearch/Discovery）に投入する際の
#     ドキュメント ID として再利用しやすい設計。
# 例外対策（コメントのみ）:
#   ・曖昧さ回避ページ（Disambiguation）や記事未存在（PageError）に備え、
#     本番は try/except でスキップ/リトライ/タイトル調整を行うのがよい。
import wikipedia

wikipedia.set_lang("ja")

data_list = (
    []
)  # 収集結果の蓄積先（リスト of 辞書）。後段のインデクシングでそのまま回せる形。
for index, title in enumerate(title_list):
    print(index + 1, title)  # 収集進捗のログ出力（長尺バッチ時の可観測性向上）
    # `auto_suggest=False`:
    #   ・類似タイトルへの自動補正を抑止し、意図しない別記事の取得を回避。
    #   ・ただしタイトルの微妙な揺れを拾えないため、失敗時は例外となる点に注意。
    text = wikipedia.page(title, auto_suggest=False).content
    # データ構造:
    #   ・app_id: 外部システムの主キーに合わせるための 1-based 連番
    #   ・title : 記事タイトル（照合/可読性のため保持）
    #   ・text  : 記事本文（wiki マークアップ除去は未実施。前処理段で正規化/品詞抽出などを行う）
    item = {"app_id": index + 1, "title": title, "text": text}
    data_list.append(
        item
    )  # ダウンストリーム（TF-IDF/同義語検証/類似検索評価）へ受け渡す基礎データとして格納

# 提示データの想定利用:
#   ・形態素解析（MeCab/Janome/kuromoji）→ トークン化・原形化・品詞フィルタ
#   ・ベクトル化（TF-IDF, BM25, sentence-transformers など）→ 特徴語抽出/類似度計算
#   ・検索基盤（Elasticsearch/Watson Discovery）→ 日本語アナライザ/同義語辞書/ユーザー辞書の効果検証
# 品質検証の観点:
#   ・「泉質語彙」「地名」「観光/歴史表現」の混在がコーパスの多様性を高め、検索の recall/precision の
#     トレードオフ評価や、辞書・同義語ルールの当たり/外れの見極めに適している。

In [ ]:
# 資格情報の設定 (個別に設定します)

discovery_credentials = {
    "apikey": "xxxx",
    "iam_apikey_description": "xxxx",
    "iam_apikey_name": "xxxx",
    "iam_role_crn": "xxxx",
    "iam_serviceid_crn": "xxxx",
    "url": "xxxx",
}

In [ ]:
# Discovery APIの初期化

import json
import os
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

version = "2019-04-30"

authenticator = IAMAuthenticator(discovery_credentials["apikey"])
discovery = DiscoveryV1(version=version, authenticator=authenticator)
discovery.set_service_url(discovery_credentials["url"])

In [ ]:
# environment_id、collection_id、configuration_id の取得
# すでにUIで1つのprivate collectionが作成済みであることが前提

# environment id の取得
environments = discovery.list_environments().get_result()["environments"]
environment_id = environments[0]["environment_id"]
if environment_id == "system":
    environment_id = environments[1]["environment_id"]
print("environment_id: ", environment_id)

# collection id の取得
collection_id = discovery.list_collections(environment_id).get_result()["collections"][
    0
]["collection_id"]
print("collection_id: ", collection_id)

# configuration idの取得
configuration_id = discovery.list_configurations(environment_id).get_result()[
    "configurations"
][0]["configuration_id"]
print("configuration_id: ", configuration_id)

In [ ]:
# 文書ロード関数
# collection_id: 対象コレクション
# sample_data: 書き込み対象テキスト (json形式の配列)
# key_name: 文書のユニークキー名称
#
# 前提条件（この関数の外側で準備が必要）:
#   - Discovery クライアント `discovery` が初期化済みであること
#   - ターゲット環境ID `environment_id` が取得済みであること
#   - 次の標準ライブラリが import 済みであること: json, os, time
#       例)
#         import json
#         import os
#         import time
#
# 設計・理論的な意図:
#   - IBM Watson Discovery のコレクションへ大量ドキュメントを一括投入する際に
#     短時間で処理キュー（processing キュー）を過負荷にしないことが重要。
#     本関数はコレクションの `document_counts.processing` を監視し、
#     閾値（ここでは 20）未満になるまで待機することでスロットリング（バックオフ）を実装している。
#     これは外部APIへの負荷制御（rate limiting/backpressure）の一種。
#   - JSON を一時ファイルとして書き出し → `add_document` にファイルオブジェクトを渡す構成。
#     Discovery はファイルアップロード型の API を提供するため、この経路が簡便。
#   - 各ドキュメントのユニークキー（`key_name`）をファイル名に用いることで、
#     ログ/監査/リトライのトレース性を確保。
#   - 実運用では:
#       * 日本語を可読のまま保存したい場合は `json.dump(..., ensure_ascii=False)` を推奨。
#       * 例外（ネットワーク障害/スロット満杯/権限エラーなど）を try/except で握り、リトライ/スキップを行う。
#       * `tempfile` モジュールで衝突しない一時ファイルを作ると安全（並列時の競合回避）。
#       * 閾値 20 は環境・プランに依存するため、経験則に応じて調整。
#
# 実装上の注意:
#   - `discovery.get_collection(...).get_result()` の戻り値は dict。
#     while ループ内の再取得時にも `.get_result()` を付け忘れると、DetailedResponse のままで
#     `[...]` による辞書アクセスが失敗するため注意（※下の該当行に注意喚起コメントを付与）。
#   - `open(..., 'w')` は明示的に `encoding='utf-8'` を与えるのが安全（環境依存の文字化け回避）。
#   - `json.dump` の既定は ASCII エスケープ。日本語をそのまま残すなら `ensure_ascii=False` を検討。
def load_text(collection_id, sample_data, key_name):
    for item in sample_data:
        # 1) アップロード対象ドキュメントの可視化（ログ出力）
        #    - デバッグ/監査のために item 全体を表示（サイズが大きい場合は要約推奨）。
        print(item)

        # 2) 一時 JSON ファイルの作成
        #    - ユニークキーをファイル名に採用し、投入対象を特定しやすくする。
        #    - 実運用では tempfile.NamedTemporaryFile の利用がより安全。
        key = item.get(key_name)  # ユニークキー（例: app_id）
        filename = str(key) + ".json"  # 例: "123.json"
        f = open(
            filename, "w"
        )  # encoding を指定しないと環境依存（UTF-8 明示が望ましい）
        json.dump(
            item, f
        )  # 日本語をエスケープせず保存: json.dump(item, f, ensure_ascii=False)
        f.close()

        # 3) コレクション処理状況の監視（バックプレッシャ制御）
        #    - processing 中のドキュメント数が多い間は待機して API への過負荷を回避。
        #    - この待機は「到着率 > 処理率」のときのキュー蓄積（待ち行列理論でいう M/M/1 等）を
        #      緩和する実装方針に準ずる。
        collection = discovery.get_collection(
            environment_id, collection_id
        ).get_result()
        proc_docs = collection["document_counts"]["processing"]

        while True:
            if proc_docs < 20:
                # 閾値未満 → 送信継続
                break
            print("busy. waiting..")
            time.sleep(10)  # ポーリング間隔。API 制限と応答遅延を見て適宜調整。

            # ※注意: 再取得時も `.get_result()` が必要（付け忘れると dict ではなく DetailedResponse になる）
            collection = discovery.get_collection(
                environment_id, collection_id
            ).get_result()  # ← .get_result() を明示
            proc_docs = collection["document_counts"]["processing"]

        # 4) JSON ファイルを Discovery にアップロード
        #    - ファイルハンドルは with 文で安全にクローズ。
        #    - add_document の戻り値（add_doc）は必要に応じてログ出力/ID 保持する。
        with open(filename) as f:
            add_doc = discovery.add_document(environment_id, collection_id, file=f)
            # ここで add_doc.get_result() を呼び、document_id や status をログするのも有用。

        # 5) 一時ファイルを削除（クリーンアップ）
        os.remove(filename)

In [ ]:
# 特定のコレクションの全文書を削除する関数
# collection_id: 対象コレクション


def delete_all_docs(collection_id):

    # 文書件数取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection["document_counts"]["available"]

    results = discovery.query(
        environment_id, collection_id, return_fields="id", count=doc_count
    ).get_result()["results"]
    ids = [item["id"] for item in results]

    for id in ids:
        print("deleting doc: id =" + id)
        discovery.delete_document(environment_id, collection_id, id)

In [ ]:
# 既存文書の全削除
delete_all_docs(collection_id)

In [ ]:
# wikipedia文書のロード
load_text(collection_id, data_list, "app_id")

In [ ]:
# リスト 4.6.14
# 定山渓温泉の id 値（Discovery が付与する内部ドキュメントID）を調べる
#
# ポイント:
# - `filter` はスコアリングに影響しない絞り込み（構造化）用の条件。高速で、結果の関連度は問わない。
# - ここでは title フィールドが「定山渓温泉」に一致する文書だけを取り出し、その最初の1件の `id` を取得する。
# - 取得する `id` は Discovery 内部のドキュメントIDであり、アプリ側で付与した `app_id` とは別物。
# - `return_fields` で返却フィールドを絞ると転送量・処理負荷を抑えられる（ここでは確認用に app_id と title のみ）。
#
# 注意点:
# - 文字列比較の安定性を高めるには、値を引用符で囲むのが無難（例: title::"定山渓温泉"）。
#   日本語や空白・記号を含む場合は特に推奨。
# - 一致する文書が0件のとき `query_results[0]` は例外になるため、実運用では件数チェックを行うこと。
#   例:
#     if not query_results:
#         raise ValueError("対象文書が見つかりませんでした")
#
# 返却フィールドの指定（app_id と title のみ返す）
return_fields = "app_id,title"

# フィルタ条件（title が「定山渓温泉」に一致）
# ※ より安全にするなら: filter_text = 'title::"定山渓温泉"'
filter_text = "title::定山渓温泉"

# 照会の実行:
# - `environment_id`, `collection_id`, `discovery` は事前に初期化済みである前提
# - `filter` は構造化フィルタ、`return_fields` は返却項目を制限
query_results = discovery.query(
    environment_id, collection_id, filter=filter_text, return_fields=return_fields
).get_result()["results"]

# 先頭ヒットの内部ドキュメントIDを取得
# （ヒット順はスコアに依存しないため、確定的な選択が必要なら別途 sort 指定や追加条件を検討）
similar_document_id = query_results[0]["id"]

# 出力（確認用）
print(similar_document_id)

In [ ]:
# リスト 4.6.15
# 類似検索と結果表示
#
# 目的:
# - 直前に特定した seed 文書（similar_document_id）に「似ている」文書を
#   IBM Watson Discovery の Similar Documents 機能で検索し、
#   各ヒットの app_id, title, スコア（類似度に基づく関連度）を表示する。
#
# 重要な前提:
# - `environment_id`, `collection_id`, `discovery` は 4.6.1〜4.6.3 で初期化済み。
# - `similar_document_id` は 4.6.14 で取得済み（コレクション内の内部ドキュメントID）。
# - コレクションには十分な文書がロードされ、インデクシングが完了していること。

# --- 類似検索の実施 ----------------------------------------------------------
# Discovery V1 の query API に以下を指定:
# - similar: 'true' を指定すると Similar Documents（既存文書に近い文書の検索）モードになる。
#   * SDK 的には bool の True でも可（例: similar=True）。ここでは元コードに合わせ 'true' を使用。
# - similar_document_ids: 類似度の基準とする seed 文書の内部 ID（配列も可だが、ここでは 1 件）。
# - 必要に応じて `similar_fields` で比較対象フィールド（例: 'text,title'）を明示できる。
#   未指定時はコレクション設定・マッピングに依存してテキストが比較される想定。
simular_results = discovery.query(
    environment_id,
    collection_id,
    similar="true",  # 類似文書検索を有効化（bool の True でも可）
    similar_document_ids=similar_document_id,  # 基準となる seed 文書 ID（文字列）
)

# レスポンス本体を取得
res = simular_results.get_result()

# 実際のヒット配列（`results`）を取り出す
# - 各要素はインデックスに保存したフィールド（app_id, title, text など）と
#   メタ情報（result_metadata）を含む辞書。
res2 = res["results"]

# --- 結果表示 ---------------------------------------------------------------
# 出力するのは:
# - app_id: 4.3/3.5 節系で投入したアプリ側の一意キー（Discovery 内部 id とは別）。
# - title: 文書タイトル。
# - score: result_metadata.score。類似度由来の関連度スコア（高いほど seed に近い）。
#
# 注意:
# - `score` は相対的指標で、閾値はユースケースに応じて調整が必要。
# - app_id や title は投入時のマッピング/データに依存するため、欠損時 KeyError となる。
#   本番では `item.get('app_id')` のように堅牢化するのが望ましい。
for item in res2:
    metadata = item["result_metadata"]  # 検索メタ情報（score など）を持つ
    score = metadata["score"]  # 類似度ベースの関連度スコア（float）
    app_id = item["app_id"]  # アプリ側で付与した一意キー（登録データ由来）
    title = item["title"]  # 文書タイトル（登録データ由来）
    print(app_id, title, score)

# --- 実運用の補足 -----------------------------------------------------------
# - 結果件数の制御: query(count=K) を追加指定してヒット数を調整。
# - フィルタ併用: similar と同時に filter= を使ってドメイン・期間・種別などで絞り込み可能。
# - 重複排除: deduplicate=True や deduplicate_field= を指定して近似重複を除去。
# - ページング: offset/sort の組み合わせで安定した反復取得を行う。
# - 監査/再現性: seed 文書 ID とパラメータ（similar_fields, filter など）を必ずログに残す。